In [3]:
##########################################################################################################
# -----------------------------------------------------------------
#  kiss_constant_verify
# -----------------------------------------------------------------
#
# INPUT:
# - a,b:   we assume the systole to lie in the interval [a,b]
# - param: a pair (Z,W), where:
#          * Z is a list of prescribed double roots for f(sqrt(x))
#          * W is a list of prescribed double roots for f^(sqrt(x))
#
# THE PROGRAM:
# - Computes a Fourier pair (f,f^) based on "genus" and "param":
#    f(x) will be of the form u(x^2)*e^(-x^2/2) for some polynomial u to be determined
#    f^(y) is v(y^2)*e^(-y^2/2) for some polynomial v to be determined
#    the degree d of u and v are determined by the total number of parameters: it's the minimal degree such
#    that there is a pair of non-zero polynomials with the prescribed values. This implies that the pair
#    (u,v) is uniquely determined by "genus" and "param"    
# - Verifies whether the pair (f,f^) satisfies the conditions of Theorem 7.1
# - If so, gives a rigorous estimate for the bound on K(s) (Definition 7.3)
#
# OUTPUT:
# - If the conditions are satisfied: the computed bound on K(s) 
# - If not, an error message
##########################################################################################################


def kiss_constant_verify(a,b,param):
    print('\n\n----------------------------------------------------------------')
    print('kiss_constant_verify')
    print('interval: [',a,',',b,']\n\n')
    
    (Z,W) = param
    
    # Make sure every input is treated as a rational number
    Z = [QQ(elem) for elem in Z]
    W = [QQ(elem) for elem in W]
    
    a = QQ(a)
    b = QQ(b)
    
    m = len(Z) #number of forced double roots for f
    n = len(W) #number of forced double zeros for f^
    
    # In order to be able to count the zeroes later on, we restrict the intervals in which they're
    # allowed to lie:
    for elem in Z:
        if elem<=b^2:
            return 'The forced double zeros for u should exceed b^2'
        
    for elem in W:
        if elem<=0:
            return 'The forced double zeros for v should be positive'

    # For convenience, we also assume that the zeros are listed in order:
    for j in range(1,m):
        if Z[j]<=Z[j-1]:
            return 'The list of forced double zeros for u is not increasing'
        
    for j in range(1,n):
        if W[j]<=W[j-1]:
            return 'The list of forced double zeros for v is not increasing'
    
    print('Checked the conditions on the prescribed zeroes')
    
    (u,v) = kiss_poly(a,param)
    
    u_der = u.derivative()
    v_der = v.derivative()

    u_der_der = u_der.derivative()
    v_der_der = v_der.derivative()
    
    print('Computed the polynomials (u,v) based on the prescribed zeroes')
    
    if u(a^2) >= 0 : # Verify that f(a) < 0
        return 'f(a) is non-negative'
        
    elif u(b^2) >= 0 : # Verify that f(b) < 0
        return 'f(b) is non-negative'
        
    elif pari.polsturm(u,[a^2,Infinity]) > m: # Verify that the number of zeroes past a^2 is no more than prescribed
        return 'f has an extra zero'
        
    elif v(-1/4) < 0: # Verify that f^(i/2) >= 0
        return 'f^(i/2) is negative'
    
    elif pari.polsturm(v,[-1/4,Infinity])>n+1: # + 1 is for the root at i/2
        return 'f^ has an extra zero'
    
    print('Verified the sign conditions on f and f^')
    
    Q.<x> = PolynomialRing(QQ)
    
    if a != b:
        # We want to prove a bound on the geometric terms in the Selberg trace formula
        # corresponding to simple geodesics whose length lies in the interval [a,b]
        # We first numerically guess a bound, essentially requiring it's given by the values at
        # the endpoints of our interval:
        def g(t):
            return t*u(t^2)*e^(-t^2/2)/(2*sqrt(2*pi)*sinh(t/2))
        upper_bound_sys_term = QQ(RDF(0.999999*max(g(a),g(b))))
        
        # To check that this gives us a bound, we use the Taylor polynomials of exp and sinh
        # Taylor polynomial of exp 
        n = 100
        E = 0
        for k in range(2*n):
            E = E + (-x^2/2)^k / factorial(k)
    
        # Taylor polynomial of sinh and its Lagrange error term at b
        S = 0
        for k in range(2*n):
            S = S + (x/2)^(2*k+1) / factorial(2*k+1)
        S_error = QQ(RBF(cosh(b/2)).upper()*(b/2)^(2*n+1) / factorial(2*n+1))
        
        testpoly = x * u(x^2)*E+2*upper_bound_sys_term*(S+S_error)

    
        if testpoly(a) >= 0 :
            return 'testpoly is non-negative at a'
        elif pari.polsturm(testpoly,[a,b]) > 0 :
            return 'testpoly is non-negatie somewhere on [a,b]'
    else:
        upper_bound_sys_term = QQ(RBF(a*u(a^2)*e^(-a^2/2)/(2*sqrt(2*pi)*sinh(a/2))).upper())
        
    
    print('Computed an upper bound on the geometric kissing term')
    
    # Compute an estimate for the integral
    # Below, RBF and CBF denote the RealBallField and ComplexBallField from the Arb package
    
    # h is defined so that f^(x) = h(x^2)
    def h(t):
        return v(t)*e^(-t/2)
    
    # We estimate the integral term in the Selberg trace formula
    truncated_integral = CBF.integral(lambda x, _: h(x^2)*x*tanh(CBF(pi)*x),0,200)
    
    print('Computed the truncated integral')
    
    # For the remainder of the integral, we compute an indefinite integral
    # We will use that x^(2*n+1) * exp(-x^2 / 2) admits a primitive
    # of the form p_n(x)*exp(-x^2 / 2), where p_n is a polynomial
    
    # First we compute the p_n's:
    max_d = v.degree() # the maximal degree for which we need to know p_n
    p = [-1] # initializing p[0] (which is p_1)
    for n in range(1,max_d):
        p += [-x^(2*n)+2*n*p[n-1]] # adding p_n to the list, the recurrence we use is obtained
                                   # from integration by parts
    
    # Compute the primitive of x*v(x^2)*exp(-x^2/2)
    V = 0
    for n in range(max_d):
        V += v.coefficients()[n]*p[n]
        
    remainder = -V.subs(x==200)*exp(-200^2/2)
    
    print('Computed a bound for the remainder of the integral')
    
    integral_fhat = truncated_integral+remainder
    
    kiss_constant_bound = RBF(2*integral_fhat/-upper_bound_sys_term).upper() 
    
    print('\nBound found: ',kiss_constant_bound)
    
    return kiss_constant_bound 
    
    


##########################################################################################################
# -----------------------------------------------------------------
#  kiss_poly
# -----------------------------------------------------------------
# INPUT:
# - a:     the lower end of the interval in which we assume the systole to lie
# - param: a pair (Z,W), where:
#          * Z is a set of prescribed double roots for f
#          * W is a set of prescribed double roots for f^
# 
# THE PROGRAM:
# - Computes a pair (u,v) of polynomials based on "genus" and "param":
#    f(x) will be of the form u(x^2)*e^(-x^2/2) for some polynomial u to be determined
#    f^(y) is v(y^2)*e^(-y^2/2) for some polynomial v to be determined
#    the degree d of u and v are determined by the total number of roots: it's the minimal degree such
#    that there is a pair of non-zero polynomials with the prescribed values. This implies that the pair
#    (u,v) is uniquely determined by "genus" and "param"    
#
# OUTPUT:
# (u,v)
##########################################################################################################

    
    
def kiss_poly(a,param):
    
    (Z,W) = param
    
    # Make sure the inputs are treated as rational numbers
    Z = [QQ(elem) for elem in Z]
    W = [QQ(elem) for elem in W]
    
    
    m = len(Z) #number of forced double roots for f
    n = len(W) #number of forced double roots for f^
    c = 2*(m+n)+2 #number of conditions imposed

    # Compile a list of Laguerre polynomials and their derivatices
    P.<x> = PolynomialRing(QQ) 
    L = [ P(gen_laguerre(n,-1/2,x)) for n in range(c)]
    L_der = [ L[n].derivative() for n in range(c)]   
    
    
    A = [[(-1)^j*L[j](W[k]) for j in range(c)] for k in range(n)] #for f^ to vanish at the w_k
    B = [[(-1)^j*L_der[j](W[k]) for j in range(c)] for k in range(n)] #for f^' to vanish at the w_k
    
    C = [[L[j](Z[k]) for j in range(c)] for k in range(m)] #for f to vanigh at the z_k
    D = [[L_der[j](Z[k]) for j in range(c)] for k in range(m)] #for f' to vanish at the z_k
    
    E = [[L[j](QQ(a^2)) for j in range(c)]]
    F = [[(-1)^j*L[j](-1/4) for j in range(c)]]
    
    M = pari(matrix(QQ,A+B+C+D+E+F)) #matrix of values of the Laguerre polynomials and their derivative or combinations thereof
    Y = pari(matrix(QQ,[0]*(c-2)+[-1]+[0]).transpose()) #this is the vector of target values
    X = vector(pari.matinverseimage(M,Y).sage())

    u = sum([X[j]*L[j] for j in range(c)]) #the polynomial for f 
    v = sum([(-1)^j*X[j]*L[j] for j in range(c)]) #the polynomial for f^
    
    return (u,v)

In [4]:
# Verify Table 2

# Load the parameters for kissing numbers
kiss_parameters = load('parameters_kissing')

kiss_constant_list = []

for p in kiss_parameters:
    a = p[0]
    b = p[1]
    param = p[2]
    kiss_constant_list += [[a,b,kiss_constant_verify(a,b,param)]]

# Print out the list of bounds on K(s)
print("\n")
for p in kiss_constant_list:
    print(p)


# Bound the maximal kissing number in genus 2 - 20
# Load the list of systoles created with the 'verify_systole' notebook
sys_list = load('list_systole')

# Create the list of half kissing numbers
kiss_list = []
for genus in range(2,21):
    j = 0
    # max_ks will be the upper bound on K(s) we found for s in the interval [1.75,sys_list[genus-2]]
    max_ks = kiss_constant_list[0][2] 
    while sys_list[genus-2] > kiss_constant_list[j][1]:
        j = j + 1
        if kiss_constant_list[j][2] > max_ks:
            max_ks = kiss_constant_list[j][2]
        
    kiss_list += [(max_ks*(genus-1)/2).floor()]

kiss_list



----------------------------------------------------------------
kiss_constant_verify
interval: [ 1.75000000000000 , 1.78000000000000 ]


Checked the conditions on the prescribed zeroes
Computed the polynomials (u,v) based on the prescribed zeroes
Verified the sign conditions on f and f^
Computed an upper bound on the geometric kissing term
Computed the truncated integral
Computed a bound for the remainder of the integral

Bound found:  27.8323475677166


----------------------------------------------------------------
kiss_constant_verify
interval: [ 1.77000000000000 , 2.29000000000000 ]


Checked the conditions on the prescribed zeroes
Computed the polynomials (u,v) based on the prescribed zeroes
Verified the sign conditions on f and f^
Computed an upper bound on the geometric kissing term
Computed the truncated integral
Computed a bound for the remainder of the integral

Bound found:  29.9252395734874


----------------------------------------------------------------
kiss_constant

Computed the polynomials (u,v) based on the prescribed zeroes
Verified the sign conditions on f and f^
Computed an upper bound on the geometric kissing term
Computed the truncated integral
Computed a bound for the remainder of the integral

Bound found:  104.277251843990


----------------------------------------------------------------
kiss_constant_verify
interval: [ 7.73000000000000 , 7.73000000000000 ]


Checked the conditions on the prescribed zeroes
Computed the polynomials (u,v) based on the prescribed zeroes
Verified the sign conditions on f and f^
Computed an upper bound on the geometric kissing term
Computed the truncated integral
Computed a bound for the remainder of the integral

Bound found:  110.150018317685


----------------------------------------------------------------
kiss_constant_verify
interval: [ 7.86353000000000 , 7.86353000000000 ]


Checked the conditions on the prescribed zeroes
Computed the polynomials (u,v) based on the prescribed zeroes
Verified the sign 

[14,
 34,
 62,
 97,
 138,
 185,
 240,
 299,
 364,
 434,
 510,
 591,
 677,
 771,
 868,
 970,
 1083,
 1209,
 1333]